In [4]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langchain.chains import RefineDocumentsChain
from langchain.chains.llm import LLMChain
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [3]:
load_dotenv()

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    max_retries=2,
)

In [5]:
document_prompt = PromptTemplate(
    input_variables=["page_content"], template="{page_content}"
)
document_variable_name = "context"
# The prompt here should take as an input variable the
# `document_variable_name`
summarize_prompt = ChatPromptTemplate(
    [
        ("human", "Write a concise summary of the following: {context}"),
    ]
)

In [6]:
initial_llm_chain = LLMChain(llm=llm, prompt=summarize_prompt)
initial_response_name = "existing_answer"

C:\Users\vedan\AppData\Local\Temp\ipykernel_6136\1564957277.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  initial_llm_chain = LLMChain(llm=llm, prompt=summarize_prompt)


In [7]:
refine_template = """
Produce a final summary.

Existing summary up to this point:
{existing_answer}

New context:
------------
{context}
------------

Given the new context, refine the original summary.
"""
refine_prompt = ChatPromptTemplate([("human", refine_template)])
refine_llm_chain = LLMChain(llm=llm, prompt=refine_prompt)

In [8]:
chain = RefineDocumentsChain(
    initial_llm_chain=initial_llm_chain,
    refine_llm_chain=refine_llm_chain,
    document_prompt=document_prompt,
    document_variable_name=document_variable_name,
    initial_response_name=initial_response_name,
    return_intermediate_steps = True
)

In [9]:
documents = TextLoader('./documents/text2.txt', encoding='utf8')

docs = documents.load()

print(len(docs))

1


In [10]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=0,
)

split_docs = text_splitter.split_documents(docs)
print(f"Generated {len(split_docs)} documents.")

Created a chunk of size 1296, which is longer than the specified 1000
Created a chunk of size 1040, which is longer than the specified 1000


Generated 13 documents.


In [11]:
print(f'Number of tokens as per the given model: - {[llm.get_num_tokens(text = i.page_content) for i in split_docs]}')
print(f'Number of characters inside each chunk: - {[len(i.page_content) for i in split_docs]}')

e:\Codes\Data Sciene\AI\Long-Text-Document-Summarization\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
e:\Codes\Data Sciene\AI\Long-Text-Document-Summarization\.venv\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (1278 > 1024). Running thi

Number of tokens as per the given model: - [981, 965, 985, 968, 1011, 976, 912, 1278, 996, 215, 1022, 1018, 313]
Number of characters inside each chunk: - [3931, 3544, 4222, 4310, 4036, 4180, 3936, 3599, 3525, 1024, 4050, 3320, 823]


In [18]:
result = chain.invoke(split_docs)

In [13]:
result

{'input_documents': [Document(metadata={'source': './documents/text2.txt'}, page_content='Table of Contents\n\n  * Agent System Overview\n  * Component One: Planning\n    * Task Decomposition\n    * Self-Reflection\n  * Component Two: Memory\n    * Types of Memory\n    * Maximum Inner Product Search (MIPS)\n  * Component Three: Tool Use\n  * Case Studies\n    * Scientific Discovery Agent\n    * Generative Agents Simulation\n    * Proof-of-Concept Examples\n  * Challenges\n  * Citation\n  * References \n\nBuilding agents with LLM (large language model) as its core controller is a\ncool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer\nand BabyAGI, serve as inspiring examples. The potentiality of LLM extends\nbeyond generating well-written copies, stories, essays and programs; it can be\nframed as a powerful general problem solver.\n\n# Agent System Overview#\n\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain,\ncomplemented by several k

In [26]:
for i in result['intermediate_steps']:
    print(i)

    print("*"*30)

Here's a concise summary of the text:

**LLM-Powered Autonomous Agent System**

Building agents with Large Language Models (LLMs) as their core controller is a promising concept. An LLM-powered autonomous agent system consists of three key components:

1. **Planning**: Breaking down complex tasks into manageable subgoals through task decomposition, and refining past actions through self-reflection.
2. **Memory**: Retaining and recalling information over extended periods using short-term and long-term memory, with the help of an external vector store and fast retrieval.
3. **Tool Use**: Learning to call external APIs for extra information, code execution capability, and access to proprietary information sources.

**Task Decomposition**

Task decomposition is crucial for planning complex tasks. Techniques like Chain of Thought (CoT), Tree of Thoughts (ToT), and LLM+P (Liu et al. 2023) are discussed, which help decompose big tasks into smaller, manageable steps.

**Key Takeaways**

* LLMs

In [20]:
print(result['output_text'])

**Refined Final Summary:**

As a {{GPT-Engineer}}, an AI assistant designed to write a comprehensive repository of code given a task specified in natural language, your primary goal is to achieve code writing, task clarification, and documentation independently, without seeking user assistance. You will leverage the strengths of your Large Language Model (LLM) and pursue simple strategies with no legal complications, adhering to best practices for the language and framework chosen.

**Key Components:**

1. **Task Clarification**: Break down complex tasks into manageable subgoals through task decomposition, and refine past instructions through self-reflection. Utilize techniques like Chain of Thought (CoT), Tree of Thoughts (ToT), and LLM+P (Liu et al. 2023) for effective task decomposition. Ensure to clarify instructions and not carry them out until they are well understood.
2. **Code Writing**: Retain and recall information over extended periods using short-term and long-term memory, 

In [29]:
for i in result['input_documents']:
    print(i)
    print("*"*50)

page_content='Table of Contents

  * Agent System Overview
  * Component One: Planning
    * Task Decomposition
    * Self-Reflection
  * Component Two: Memory
    * Types of Memory
    * Maximum Inner Product Search (MIPS)
  * Component Three: Tool Use
  * Case Studies
    * Scientific Discovery Agent
    * Generative Agents Simulation
    * Proof-of-Concept Examples
  * Challenges
  * Citation
  * References 

Building agents with LLM (large language model) as its core controller is a
cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer
and BabyAGI, serve as inspiring examples. The potentiality of LLM extends
beyond generating well-written copies, stories, essays and programs; it can be
framed as a powerful general problem solver.

# Agent System Overview#

In a LLM-powered autonomous agent system, LLM functions as the agent’s brain,
complemented by several key components:

  * **Planning**
    * Subgoal and decomposition: The agent breaks down large tasks int

In [38]:
# Count tokens for each document chunk
input_tokens = [llm.get_num_tokens(doc.page_content) for doc in split_docs]
total_input_tokens = sum(input_tokens)
print(f"Total input tokens: {total_input_tokens}")

Total input tokens: 11640


In [39]:
# Count tokens for the initial summary
initial_summary = result["intermediate_steps"][0]
initial_summary_tokens = llm.get_num_tokens(initial_summary)
print(f"Initial summary tokens: {initial_summary_tokens}")


Initial summary tokens: 297


In [40]:
# Count tokens for each refinement step
refinement_tokens = [llm.get_num_tokens(step) for step in result["intermediate_steps"][1:]]
total_refinement_tokens = sum(refinement_tokens)
print(f"Total refinement tokens: {total_refinement_tokens}")


Total refinement tokens: 16010


In [41]:
# Count tokens for the final output
final_output_tokens = llm.get_num_tokens(result["output_text"])
print(f"Final output tokens: {final_output_tokens}")


Final output tokens: 2450


In [42]:
# Count tokens for the summarize prompt template
summarize_prompt_template = summarize_prompt.format_prompt(context="sample").to_string()
summarize_prompt_tokens = llm.get_num_tokens(summarize_prompt_template)
print(f"Summarize prompt tokens: {summarize_prompt_tokens}")

# Count tokens for the refine prompt template
refine_prompt_template = refine_prompt.format_prompt(existing_answer="sample", context="sample").to_string()
refine_prompt_tokens = llm.get_num_tokens(refine_prompt_template)
print(f"Refine prompt tokens: {refine_prompt_tokens}")

Summarize prompt tokens: 11
Refine prompt tokens: 46


In [43]:
# Calculate total tokens including templates
total_prompt_tokens = summarize_prompt_tokens + refine_prompt_tokens

# Total tokens used in the process
total_tokens_with_prompts = (total_input_tokens + total_refinement_tokens +
                              final_output_tokens + total_prompt_tokens)

print(f"Total input tokens: {total_input_tokens}")
print(f"Total intermediate tokens: {total_refinement_tokens}")
print(f"Total prompt tokens: {total_prompt_tokens}")
print(f"Total tokens used (including prompts): {total_tokens_with_prompts}")


Total input tokens: 11640
Total intermediate tokens: 16010
Total prompt tokens: 57
Total tokens used (including prompts): 30157
